# **IMPORTS**

In [ ]:
! pip -q install transformers
import pandas as pd
import json
import os
import shutil
import numpy as np
import torch
#torch.cuda.empty_cache()
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2DoubleHeadsModel, GPT2LMHeadModel, GPT2Model,AutoModelForCausalLM, AutoTokenizer,GPT2Model
from tqdm.notebook import tqdm
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')
drive.mount('/content/drive')
with open('/content/drive/MyDrive/personachat_self_original.json', "r", encoding="utf-8") as f:
    dataset = json.loads(f.read())
print(type(dataset))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
Mounted at /content/drive
<class 'dict'>


# **PREPROCESSING**

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("microsoft/DialoGPT-medium")
#model2 = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/modello')
#tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
model = GPT2Model.from_pretrained("microsoft/DialoGPT-medium")

special_tokens = ['<SP1>', '<SP2>', '<PAD>']
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})
special_ids = tokenizer.convert_tokens_to_ids([
        '<SP1>', '<SP2>', tokenizer.bos_token,
        tokenizer.eos_token,  '<PAD>'])
model.resize_token_embeddings(len(tokenizer))

personality=[]
utterances=[]
#for i in range(len(dataset['train'])):
for i in range(100):
  personality.append(dataset['train'][i]['personality'])
  utterances.append(dataset['train'][i]['utterances'])
persona=[]
history=[]
spokerA=[]
spokerB_real=[]
spokerB_distract1=[]
for i in range(len(personality)-1):
  for j in range(len(utterances[i])):
    candidate = utterances[i][j]['candidates']
    history1 = utterances[i][j]['history']
    persona.append(personality[i])
    history.append(history1)
    spokerA.append(history1[-1])
    spokerB_real.append(candidate[-1])
    spokerB_distract1.append(candidate[-2])


def make_input(persona, spokerA, spokerB):
  input=[]
  for i in range(len(persona)):
    ids=[]

    spA='<SP1> '+spokerA[i]+tokenizer.eos_token
    spB='<SP2> '+spokerB[i]+tokenizer.eos_token
    ids.append(spA)
    ids.append(spB)
    input.append(ids)

  return input

input1=make_input(persona,spokerA,spokerB_real)
input_fake=make_input(persona,spokerA,spokerB_distract1)


def unifica_coppie(array):
    nuovo_array = []
    for i in range(0, len(array)-1, 2):
        coppia1 = array[i]
        coppia2 = array[i+1]
        elemento_unificato = coppia1 + coppia2
        nuovo_array.append(elemento_unificato)
    return nuovo_array

input2=unifica_coppie(input1)
input2_fake=unifica_coppie(input_fake)

print("Length input2: ",len(input2))

def unifica_array(array):
    nuovo_array = []
    for sublist in array:
        nuovo_array.extend(sublist)
    return nuovo_array

def make_input_ids(input):
  encoded_input=[]
  for x in input:
    for j in x:
      sentence=[]
      leng=0
      for y in j:
        leng=len(j)
        y=tokenizer.encode(y,truncation=True,max_length=512)
        sentence.append(y)
      sentence.append(tokenizer.encode('<PAD>')*(256-leng))
      sentence=unifica_array(sentence)
      if(len(sentence)<=256):
        encoded_input.append(sentence)
  return torch.tensor(encoded_input)

encoded_input=make_input_ids(input2)
encoded_input_fake=make_input_ids(input2_fake)

cls_label_R = torch.ones(size=(encoded_input.size(0),1), dtype=torch.long)
cls_label_F1 = torch.zeros(size=(encoded_input_fake.size(0),1), dtype=torch.long)

def concat_input(input_ids1, input_ids2):
  return torch.cat((input_ids1, input_ids2), dim=0)

fin_input = concat_input(encoded_input, encoded_input_fake)
fin_label = concat_input(cls_label_R , cls_label_F1)

"""print(fin_input[0])
print(fin_label)
"""
print(tokenizer.encode('<PAD>')) #[50259]
print(tokenizer.encode('<SP1>')) #[50257]
print(tokenizer.encode('<SP2>')) #[50258]
print(tokenizer.eos_token_id) #50256

def make_token_type_ids(input_str):
  x=0
  #input_str=input_str.view(-1)
  tok=[]
  for i in range(len(input_str)):
    if input_str[i]==50257:
      x=1
    if input_str[i]==50258:
      x=2
    #if input_str[i]==50259:
    #  x=3
    tok.append(x)

  return torch.tensor(tok)

def make_LMlabel(input_ids):
    input=input_ids.squeeze()
    label=torch.zeros(size=input.size(), dtype=torch.long)
    for i in range(len(input)-1):
      label[i]=input[i+1]
    if input[len(input)-1] == 50259:
      label[len(input)-1]=50259
    else: label[len(input)-1]=50256

    return label.unsqueeze(0)

Some weights of the model checkpoint at microsoft/DialoGPT-medium were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Length input2:  363
[50259]
[50257]
[50258]
50256


# **DATALOADER**

In [ ]:
#crea un dataset personalizzato per l'addestramento del modello, prende in input input_ids(identificatori di input)  e
#cls_label (etichette di classificazione)

class Custom_Dataset(Dataset):
  def __init__(self, input_ids,cls_label):
    self.input_ids = input_ids
    self.cls_label = cls_label
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    input_ids = self.input_ids[idx]
    token_type_ids = make_token_type_ids(self.input_ids[idx])
    LM_label = make_LMlabel(self.input_ids[idx])
    cls_label = self.cls_label[idx]

    return input_ids, token_type_ids, LM_label, cls_label

chat_dataset = Custom_Dataset(fin_input, fin_label)

Batch_size=4
dataloader = DataLoader(chat_dataset, batch_size=Batch_size, shuffle=True,drop_last=True)

# **MODEL**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device: {}".format(device))
class chatbot_model(nn.Module):
  def __init__(self):
    super(chatbot_model, self).__init__()
    self.model = model
    self.drop = nn.Dropout(0.2)
    self.line = nn.Linear(1024*2,1)
    self.LM=nn.Linear(1024,50260)
    #self.LM = nn.Linear(1024,50260) #output del language modeling (con funzione di attivazione softmax) restituisce un tensore
    # con dimensione finale 50250 che è il numero di token nel vocabolario per il language modeling(si restituisce quello con prob. maggiore)
    self.m = nn.Sigmoid() #sigmoide non serve perchè è usata dalla loss function per la classificazione (come la cross entropy per il LM)

  def forward(self, input,token):
    out = self.model(input,token_type_ids= token) #tensore che rappresenta le rappresentazioni latenti delle parole di input
    x = out[0]
    x = self.drop(x)

    #print("x.size: ",x.size())
    apool=torch.mean(x,1)
    mpool,_=torch.max(x,1)
    """print("apool.size: ",apool.size())
    print("mpool.size: ",mpool.size())"""
    catcat=torch.cat((apool,mpool),1)
    #print("catcat.size: ",catcat.size())
    catcat = self.line(catcat) #fornisce un valore di output che ci da la probabilità che la risposta generata sia un distrattore o sia corretta
    x = self.LM(x)
    return x, catcat

chatbot=chatbot_model() #model
chatbot=chatbot.to(device)

Device: cuda


# **TRAIN**

In [ ]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('always')

lr = 2e-5

"""chatbot=chatbot_model()
chatbot=chatbot.to(device)"""

def train_one_epoch(data_loader, model, optimizer, device):

  model.train()
  tk0 = tqdm(data_loader, total=len(data_loader))#barra di avanzamento durante le iterazioni del training
  total_loss = 0.0

  for bi, d in enumerate(tk0):#iterazioni attraverso data_loader
      input_ids, token_type_ids, LM_label, CLS_label = d #estrae i tensori di input dal batch
      input_ids = input_ids.to(device, dtype=torch.long) #id degli input
      token_type_ids = token_type_ids.to(device, dtype=torch.long)
      LM_label = LM_label.to(device, dtype=torch.long)
      CLS_label = CLS_label.to(device, dtype=torch.float) #etichette per l'addestramento del modello di classificazione

      model.zero_grad() #azzeramento dei gradienti prima di eseguire il passaggio all'indietro per evitare che si accumulino
      LM_logits, CLS_logits = model(input_ids, token_type_ids) #esegue il forward del modello e restituisce i logits per il LM e il CLS
      LM_logits = LM_logits.view(-1, 50260)
      LM_label = LM_label.view(-1)
      #loss_LM = nn.CrossEntropyLoss()(LM_logits, LM_label) #loss del modello di linguaggio
      loss_LM=loss_LM = nn.CrossEntropyLoss()(LM_logits, LM_label)
      loss_CLS = nn.BCEWithLogitsLoss()(CLS_logits, CLS_label)*2 #loss per la classificazione
      #BCEWithLogitsLoss funzione di loss comunemente usata per problemi di classificazione binaria, questa loss combina la funzione sigmoide
      #e la loss di entropia binaria. Accetta due parametri,  i logits predetti e le etichette reali. I logits sono i valori di output prodotti
      # prima dell'attivazione, le etichette reali sono valori binari (0,1) che indicano le classi di riferimento per l'addestramento

      loss = loss_LM + loss_CLS #calcolo della loss totale combinando le due loss
      total_loss += loss.item() #aggiunge il valore della loss totale al valore accumulato
      """if bi % 100 ==0: #stampa il valore della loss ogni 100 batch
          print(f"loss:{loss}")
      if bi % 500 ==0: # monitorazione delle loss separate
          print(f"LM_loss:{loss_LM}")
          print(f"CLS_loss:{loss_CLS}")"""
      #per entrambe le loss esegui il passaggio all'indietro per calcolare i gradienti rispetto ai pesi del modello
      loss_LM.backward(retain_graph=True)
      loss_CLS.backward()
      optimizer.step() #aggiorna i pesi del modello in modo da ridurre la loss
      optimizer.zero_grad() #azzeramento dei gradienti

  avg_train_loss = total_loss / len(data_loader)
  print(" Average training loss: {0:.2f}".format(avg_train_loss))

#funzione fit per addestrare il modello
def fit(train_dataloader, EPOCHS=1):
  optimizer = torch.optim.AdamW(chatbot.parameters(),lr=lr) #optimizer

  for i in range(EPOCHS):
    print(f"EPOCHS:{i+1}")
    print('TRAIN')
    train_one_epoch(train_dataloader, chatbot, optimizer, device)
    torch.save(chatbot ,"transfo_output"+f'model:{i+1}') #dopo aver completato un epoca, il modello viene salvato


# **START TRAINING**

In [ ]:
fit(dataloader)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


EPOCHS:1
TRAIN


100%|██████████| 726/726 [15:19<00:00,  1.27s/it]


 Average training loss: 2.71


# **POSTPROCESSING**

In [ ]:
# implementazione del cahtbbot per generare risposte in base alla personaliotà e alla cronologia della conversazione fornite
# il chatbot genera testo telezionando i token successivi in base alle probabilità predette dal modello, viene anche
# calcolata una previsione di classificaizone binaria per valutare la coerenza delle risposte generate
# l'interazione avviene attraverso la funzione interact

def make_input_chat(input): #prende una lista di input e li converte in formato adatto all'interazione con il chatbot
  input_ids = []
  ids = []
  for j in range(len(input)):
      ids += tokenizer.encode(input[j])
  ids = tokenizer.encode(tokenizer.bos_token)+ids+[50258]
  input_ids.append(ids)
  return torch.tensor(input_ids)

def choose_from_top(probs, n=5): #restituisce l'indice del token successivo selezionato casualmente dai primi n token
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

def find_num(aa,num): #restituisce il numero di occorrenze del numero (num) all'interno del tensore (aa), serve per contare il numero di
                      #occorrenze di token speciali
  s=0
  for i in range(len(aa.view(-1))):
    aa=aa.view(-1)
    #print(aa.cpu().detach().numpy().tolist()[i])

    if (aa.cpu().detach().numpy().tolist()[i]) == (num):

      s=s+1

  return s

def generate_some_text(input_str,token_type_ids, model): #general del testo utilizzando un modello di chatbot
    #prende in input una stringa codificata, i token di tipo, e il modello
    #utilizza un ciclo while finchè non raggiunge un numero specificato di token di fine sequenza, durante la generazione
    # utilizza il modello per ottenere le previsioni dei token succerssivi e sceglie casualmente tra i primi n token con prob più elevate
    # aggiunge i token al tensore e continua fino alla condizione di terminazione

    cur_ids = input_str
    length = (cur_ids.size(-1))
    model.eval()
    with torch.no_grad():

      while find_num(cur_ids, 50259)<3:

            outputs = model(cur_ids, token_type_ids)
            LM_logits, CLS_logits = outputs
            softmax_logits = torch.softmax(LM_logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=5) #Randomly(from the given probability distribution) choose the next word from the top n words

            if next_token_id == 50259:
              token_type_ids= torch.cat([token_type_ids, torch.ones((1,1)).long().to(device)*2], dim=1)
            else:
              if find_num(cur_ids, 50258)>find_num(cur_ids, 50257):
                if next_token_id == 50258:
                  next_token_id = 50257
                  token_type_ids= torch.cat([token_type_ids, torch.ones((1,1)).long().to(device)*1], dim=1)
                else:
                  token_type_ids= torch.cat([token_type_ids, torch.ones((1,1)).long().to(device)*2], dim=1)

              elif  find_num(cur_ids, 50258)<=find_num(cur_ids, 50257):
                if next_token_id == 50257:
                  next_token_id = 50258
                  token_type_ids= torch.cat([token_type_ids, torch.ones((1,1)).long().to(device)*2], dim=1)
                else: token_type_ids= torch.cat([token_type_ids, torch.ones(1,1).long().to(device)*1], dim=1)

            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

            if cur_ids.size(-1)-length>30:
              break



      output_list = list(cur_ids.squeeze().to('cpu').numpy())
      output_text = tokenizer.decode(output_list)

      return output_text, cur_ids, token_type_ids, length

def cls_some_text(cur_ids, token_type_ids, model, list_cls): #calcoal la previsione di classificazione binaria del modello
    #per il testo generato fino a quel punto, calcola le previsioni utilizzando il modello e la funzione di attivazione sigmoidale

    model.eval()
    with torch.no_grad():
      _, cls_outputs = model(cur_ids, token_type_ids)
      cls = nn.Sigmoid()(cls_outputs)
      list_cls.append(cls)

    return list_cls

def chat(personality, history, tokenizer, model=chatbot, num=3): # gestisce l'intera conversazione, prende una lista di personalità,
    #una cronologia, un tokenizer e un modello di chatbot. Prepara l'input concatenando queste cose, quindi chiama make_input_chat e
    # make_token_type_ids per ottenere i tensori di input e i toke di tipo. Utilizzzzzzza il modello per generare testi e calcola le previsioni
    # di classificazione per ognuno di essi, restituisce il testo generato con la previsione di CLS più alta.

  #input_str 토큰화 하기
  speechs = []
  list_cls = []
  input=[]
  for j in range(len(personality)):
      input.append(personality[j])
  for k in range(len(history)):
      if k ==0:
        input.append('<SP1> '+history[k])
      else: input.append(history[k])

  input_str = make_input_chat(input)
  input_str2=input_str.view(-1)
  token_type_ids = make_token_type_ids(input_str2)
  input_str = input_str.to(device)
  token_type_ids = token_type_ids.view(1,-1).to(device)

  for i in range(num):
    o, c, t, l = generate_some_text(input_str,token_type_ids, chatbot)
    speechs.append(o)
    list_cls = cls_some_text(c, t, chatbot, list_cls)

  """print(type(list_cls))
  print(type(list_cls[0]))
  print(len(list_cls))
  print(list_cls[0])"""
  list_cls = np.array([tensor.item() for tensor in list_cls])
  idx = np.argmax(list_cls)
  speech = speechs[idx]

  return speech, l

def interact(personality, model): # permette l'interazione dell utente con il chatbot
  history = []
  #chatbo=chatbot_model()
  #chatbo=chatbo.to(device)
  model = torch.load("output/model.pt")
  while True:
        raw_text = input(">>> ")
        while not raw_text:
            print('Prompt should not be empty!')
            raw_text = input(">>> ")
        history.append(raw_text)
        with torch.no_grad():
            out_ids, l = chat(personality, history, tokenizer, model)
        out = tokenizer.encode(out_ids)
        spoker = out[l:]
        out_text = tokenizer.decode(out, skip_special_tokens=True)
        spoker_text = tokenizer.decode(spoker, skip_special_tokens=True)
        history.append(spoker_text)

        #print(out_text)
        print(spoker_text)

personal = ['i like to remodel homes .',
 'i like to go hunting .',
 'i like to shoot a bow .',
 'my favorite holiday is halloween .']

#interact(personal, chatbot)

# **INTERAZIONE**

In [ ]:
interact(personal, chatbot)